In [2]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab, build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Subset
from collections import Counter
import os
import spacy
import csv
from sklearn.model_selection import train_test_split

In [3]:
! spacy download en_core_web_lg

2023-07-20 06:49:31.419814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
nlp = spacy.load("en_core_web_lg")

In [5]:
 ! pip install -q kaggle

In [6]:
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus

100% 799M/799M [00:34<00:00, 31.4MB/s]
100% 799M/799M [00:34<00:00, 24.5MB/s]


In [7]:
! unzip ubuntu-dialogue-corpus.zip

Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


In [8]:
def tokenizer(text):
    return [tok.text.strip() for tok in nlp.tokenizer(text) if not tok.is_punct]

In [9]:

question_answer_pairs = []
with open('Ubuntu-dialogue-corpus/dialogueText.csv', newline='') as csvfile:
    dialogue = csv.reader(csvfile)
    question = ''
    answer = ''
    previous_frm = 'Na'
    previous_to = 'Na'
    for idx, row in enumerate(dialogue):
      # skip the first row
      if idx ==0:
        continue


      frm = row[3]
      to = row[4]
      text = row[5]

      #TODO this logic doesn't account for multiple answers to the samae question

      # if the text is to the previous from it's an answer
      if to == previous_frm:
        answer += f' {text}'
      # if the from and to are still the same it's the sam question
      elif frm == previous_frm and to == previous_to:
        question += f' {text}'
      # if the from or to are different we assume it's a new question
      elif frm != previous_frm or to != previous_to:
        # if the previous question got an answer add it

        if answer and len(question.split(' ')) < 20:
          question_answer_pairs.append((question, answer))
        question = text
        answer = ''
      else:
        raise ValueError(f'unknown state: {frm} - {to} - {previous_frm} -- {previous_to}')

      previous_frm = frm
      previous_to = to



In [10]:


def question_iter():
  for question, _ in question_answer_pairs:
    yield tokenizer(question)

def answer_iter():
  for _, answer in question_answer_pairs:
    yield tokenizer(answer)

vocab_q = build_vocab_from_iterator(question_iter(), specials=['<pad>','<bos>','<eos>'])
vocab_a = build_vocab_from_iterator(answer_iter(), specials=['<pad>','<bos>','<eos>'])

In [11]:
max(vocab_q.get_stoi().values())

87316

In [12]:
data = []
for question, answer in question_answer_pairs:
  question_tensor_ = torch.tensor([vocab_q[token] for token in tokenizer(question)],
                            dtype=torch.long)
  answer_tensor_ = torch.tensor([vocab_a[token] for token in tokenizer(answer)],
                            dtype=torch.long)
  data.append((question_tensor_, answer_tensor_))

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

BATCH_SIZE = 64
PAD_IDX = vocab_a['<pad>']
BOS_IDX = vocab_a['<bos>']
EOS_IDX = vocab_a['<eos>']

def generate_batch(data_batch):
  batch_question, batch_answer = [], []
  for (question, answer) in data_batch:
    batch_answer.append(torch.cat([torch.tensor([BOS_IDX]), answer, torch.tensor([EOS_IDX])], dim=0))
    batch_question.append(torch.cat([torch.tensor([BOS_IDX]), question, torch.tensor([EOS_IDX])], dim=0))
  batch_answer = pad_sequence(batch_answer, padding_value=PAD_IDX)
  batch_question = pad_sequence(batch_question, padding_value=PAD_IDX)
  return batch_question, batch_answer

train_idx, test_idx = train_test_split(list(range(len(data))), test_size=0.1)
train_idx, val_idx = train_test_split(train_idx, test_size=0.1)


train_iter = DataLoader(Subset(data, train_idx), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(Subset(data, test_idx), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
val_iter = DataLoader(Subset(data, val_idx), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)

In [14]:
from binascii import Error
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        print(f'input embedding {input_dim} {emb_dim}')
        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:
        try:
          embedded = self.dropout(self.embedding(src))

          outputs, hidden = self.rnn(embedded)

          hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

          return outputs, hidden
        except Error as e:
          print(e)
          raise


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        print(f'output embedding {output_dim} {emb_dim}')
        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(vocab_q)
OUTPUT_DIM = len(vocab_a)
#ENC_EMB_DIM = 256
#DEC_EMB_DIM = 256
#ENC_HID_DIM = 512
#DEC_HID_DIM = 512
#ATTN_DIM = 64
#ENC_DROPOUT = 0.5
#DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

input embedding 87317 32
output embedding 63280 32
The model has 19,147,928 trainable parameters


In [15]:
print(INPUT_DIM)

87317


In [16]:
PAD_IDX = vocab_a['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
import math
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/model.data')